In [12]:
import torch
from ucimlrepo import fetch_ucirepo 

from neural_blueprints.architectures import MLP
from neural_blueprints.config.architectures import MLPConfig
from neural_blueprints.config.utils import TrainerConfig
from neural_blueprints.config.components.composite.projections.input import TabularInputProjectionConfig
from neural_blueprints.config.components.composite.projections.output import TabularOutputProjectionConfig
from neural_blueprints.utils import Trainer, infer_types, accuracy
from neural_blueprints.preprocess import TabularPreprocessor
from neural_blueprints.datasets import MaskedTabularDataset, TabularSingleLabelDataset

import logging
logging.basicConfig(
    level=logging.DEBUG,  # or DEBUG if you want even more detail
    format="%(asctime)s - %(name)s - %(levelname)s - %(message)s"
)

In [13]:
data = fetch_ucirepo(id=2)
X = data.data.features
y = data.data.targets

data = X.copy()
data['income'] = y['income'].str.replace('.', '')     # Some target values have a trailing dot

dtypes = infer_types(data)
data = data.astype(dtypes)
data.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [14]:
preprocessor = TabularPreprocessor()
data, discrete_features, continuous_features = preprocessor.run(data)
data.head()

2025-12-22 12:14:49,660 - neural_blueprints.preprocess.tabular_preprocess - INFO - Identified 10 discrete features: ['workclass', 'education', 'education-num', 'marital-status', 'occupation', 'relationship', 'race', 'sex', 'native-country', 'income']
2025-12-22 12:14:49,660 - neural_blueprints.preprocess.tabular_preprocess - INFO - Identified 5 continuous features: ['age', 'fnlwgt', 'capital-gain', 'capital-loss', 'hours-per-week']


,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
0,0.301370,8,0.044131,10,5,5,2,2,5,2,0.02174,0.0,0.397959,40,1
1,0.452055,7,0.048052,10,5,3,5,1,5,2,0.00000,0.0,0.122449,40,1
2,0.287671,5,0.137581,12,16,1,7,2,5,2,0.00000,0.0,0.397959,40,1
3,0.493151,5,0.150486,2,14,3,7,1,3,2,0.00000,0.0,0.397959,40,1
4,0.150685,5,0.220635,10,5,3,11,6,3,1,0.00000,0.0,0.397959,6,1


### Income Inference Accuracy

In [15]:
dataset = TabularSingleLabelDataset(
    data=data,
    label_column='income',              # Specify the label column for single-label classification
    discrete_features=discrete_features,
    continuous_features=continuous_features
)

train_size = int(0.9 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size])

In [16]:
# Define model configuration
mlp_config = MLPConfig(
    hidden_dims=[128, 64, 32, 16],
    output_dim=3,
    normalization="batchnorm1d",
    activation='gelu',
    final_activation="softmax",
    input_projection=TabularInputProjectionConfig(
        cardinalities=dataset.cardinalities,
        hidden_dims=[64, 32],
        output_dim=16,
        normalization="batchnorm1d",
        activation="gelu"
    )
)

# Initialize model
model = MLP(mlp_config)
model.blueprint()

2025-12-22 12:14:49,891 - neural_blueprints.architectures.mlp - INFO - Using input projection: TabularInputProjection


Sequential(
  (0): TabularInputProjection(
    (input_projections): ModuleList(
      (0): FeedForwardNetwork(
        (network): Sequential(
          (0): DenseLayer(
            (layer): Sequential(
              (0): Linear(in_features=1, out_features=64, bias=True)
              (1): NormalizationLayer(
                (network): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
              )
              (2): GELU(approximate='none')
              (3): DropoutLayer(
                (dropout): Dropout(p=0.0, inplace=False)
              )
            )
          )
          (1): DenseLayer(
            (layer): Sequential(
              (0): Linear(in_features=64, out_features=32, bias=True)
              (1): NormalizationLayer(
                (network): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
              )
              (2): GELU(approximate='none')
              (3): DropoutLayer(
                (dropo

MLPConfig(input_dim=224, hidden_dims=[128, 64, 32, 16], output_dim=3, normalization='batchnorm1d', activation='gelu', final_activation='softmax', input_projection=TabularInputProjectionConfig(cardinalities=[1, 10, 1, 16, 16, 7, 16, 6, 5, 2, 1, 1, 1, 43], hidden_dims=[64, 32], output_dim=16, dropout_p=None, normalization='batchnorm1d', activation='gelu'), output_projection=None)

In [17]:
trainer = Trainer(
    config=TrainerConfig(
        optimizer="adam",
        criterion="cross_entropy",
        learning_rate=1e-3,
        weight_decay=1e-5,
        batch_size=128,
        early_stopping_patience=5,
        save_weights_path="./models/mlp_adult.pth"
    ),
    model= model
)

# Train the model
trainer.train(train_dataset, val_dataset, epochs=20)

2025-12-22 12:14:49,909 - neural_blueprints.utils.trainer - INFO - Trainer initialized on device: cpu


Directory ./models already exists. Existing weights are overwritten.


Training Epochs:   5%|▌         | 1/20 [00:06<02:07,  6.72s/epoch]

Epoch 1/20, Training Loss: 0.9576, Validation Loss: 0.9316


Training Epochs:  10%|█         | 2/20 [00:13<02:01,  6.77s/epoch]

...

Training Epochs:  40%|████      | 8/20 [00:57<01:29,  7.49s/epoch]

Epoch 8/20, Training Loss: 0.9239, Validation Loss: 0.9306


Training Epochs:  45%|████▌     | 9/20 [01:04<01:20,  7.30s/epoch]

Epoch 9/20, Training Loss: 0.9235, Validation Loss: 0.9286


2025-12-22 12:16:01,404 - neural_blueprints.utils.trainer - INFO - No improvement in validation loss for 5 consecutive epochs. Early stopping at epoch 10.
Training Epochs:  45%|████▌     | 9/20 [01:11<01:27,  7.94s/epoch]
2025-12-22 12:16:01,406 - neural_blueprints.utils.trainer - INFO - Training completed in 71.50 seconds.
2025-12-22 12:16:01,407 - neural_blueprints.utils.trainer - INFO - Best validation loss: 9.2675e-01


In [24]:
X = torch.tensor(val_dataset[:][0])
y = val_dataset[:][1]
with torch.no_grad():
    y_pred = model(X)
    y_pred = y_pred.argmax(dim=1)
print(f"Predictions: {y_pred[:5]}, \n Ground Truth: {y[:5]}")
acc = accuracy(y_pred, y)
print(f"Validation Accuracy: {acc:.4f}")

Predictions: tensor([2, 2, 2, 1, 1]), 
 Ground Truth: tensor([2, 2, 2, 1, 1])
Validation Accuracy: 0.8549


C:\Users\jager\AppData\Local\Temp\ipykernel_24796\3930248022.py:1: UserWarning:

To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).



### Masked Dataset Inference Accuracy

In [7]:
dataset = MaskedTabularDataset(
    data = data,
    discrete_features = discrete_features,
    continuous_features = continuous_features,
    mask_prob=0.35
)

train_size = int(0.9 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size])

In [8]:
# Define model configuration
mlp_config = MLPConfig(
    hidden_dims=[128, 64, 32, 16],
    normalization="batchnorm1d",
    activation='gelu',
    final_activation="relu",
    input_projection=TabularInputProjectionConfig(
        cardinalities=dataset.cardinalities,
        hidden_dims=[64, 32],
        output_dim=16,
        normalization="batchnorm1d",
        activation="gelu"
    ),
    output_projection=TabularOutputProjectionConfig(
        cardinalities=dataset.cardinalities,
        input_dim=16,
        hidden_dims=[8],
        activation="gelu",
        normalization="batchnorm1d",
        final_activation=None
    )
)

# Initialize model
model = MLP(mlp_config)
model.blueprint()

2025-12-22 12:13:37,552 - neural_blueprints.architectures.mlp - INFO - Using input projection: TabularInputProjection
2025-12-22 12:13:37,561 - neural_blueprints.architectures.mlp - INFO - Using output projection: TabularOutputProjection


Sequential(
  (0): TabularInputProjection(
    (input_projections): ModuleList(
      (0): FeedForwardNetwork(
        (network): Sequential(
          (0): DenseLayer(
            (layer): Sequential(
              (0): Linear(in_features=1, out_features=64, bias=True)
              (1): NormalizationLayer(
                (network): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
              )
              (2): GELU(approximate='none')
              (3): DropoutLayer(
                (dropout): Dropout(p=0.0, inplace=False)
              )
            )
          )
          (1): DenseLayer(
            (layer): Sequential(
              (0): Linear(in_features=64, out_features=32, bias=True)
              (1): NormalizationLayer(
                (network): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
              )
              (2): GELU(approximate='none')
              (3): DropoutLayer(
                (dropo

MLPConfig(input_dim=240, hidden_dims=[128, 64, 32, 16], output_dim=240, normalization='batchnorm1d', activation='gelu', final_activation='relu', input_projection=TabularInputProjectionConfig(cardinalities=[1, 10, 1, 16, 16, 7, 16, 6, 5, 2, 1, 1, 1, 43, 2], hidden_dims=[64, 32], output_dim=16, dropout_p=None, normalization='batchnorm1d', activation='gelu'), output_projection=TabularOutputProjectionConfig(cardinalities=[1, 10, 1, 16, 16, 7, 16, 6, 5, 2, 1, 1, 1, 43, 2], input_dim=16, hidden_dims=[8], activation='gelu', normalization='batchnorm1d', dropout_p=None, final_activation=None))

In [9]:
trainer = Trainer(
    config=TrainerConfig(
        optimizer="adam",
        criterion="mixed_type_reconstruction_loss",
        learning_rate=1e-3,
        weight_decay=1e-5,
        batch_size=128,
        early_stopping_patience=5,
        save_weights_path="./models/mlp_adult.pth"
    ),
    model= model
)

# Train the model
trainer.train(train_dataset, val_dataset, epochs=20)

2025-12-22 12:13:37,586 - neural_blueprints.utils.trainer - INFO - Trainer initialized on device: cpu


Directory ./models already exists. Existing weights are overwritten.


Training Epochs:   0%|          | 0/20 [00:05<?, ?epoch/s]


KeyboardInterrupt: 

In [ ]:
X = val_dataset[:][0]
y = val_dataset[:][1]
mask = val_dataset[:][2]
with torch.no_grad():
    y_pred = model(x=X)

dis_accuracy = 0
cont_accuracy = 0
for column_idx, column_name in enumerate(data.columns):
    print(f"\nFeature Column {column_name}:")
    predicted_attributes = y_pred[column_idx]      # shape: (batch_size, num_classes)
    targets = y[:, column_idx]                     # shape: (batch_size,)

    feature_mask = mask[:, column_idx]                  # shape: (batch_size,)
    predicted_attributes = predicted_attributes[feature_mask]
    if predicted_attributes.size(1) > 1:
        predicted_attributes = predicted_attributes.softmax(dim=-1).argmax(dim=-1).cpu().numpy()
    else:
        predicted_attributes = predicted_attributes.squeeze(-1).cpu().numpy()
    targets = targets[feature_mask].cpu().numpy()

    print("Predicted attribute values:", predicted_attributes[:5])
    print("True attribute values:", targets[:5])

    accuracy_value = accuracy(torch.tensor(predicted_attributes), torch.tensor(targets))
    print(f"Accuracy: {accuracy_value:.4f}")
    if column_name in discrete_features:
        dis_accuracy += accuracy_value
    else:
        cont_accuracy += accuracy_value

avg_dis_accuracy = dis_accuracy / len(discrete_features) if len(discrete_features) > 0 else 0
avg_cont_accuracy = cont_accuracy / len(continuous_features) if len(continuous_features) > 0 else 0
print(f"\nAverage Discrete Accuracy: {avg_dis_accuracy:.4f}")
print(f"Average Continuous Accuracy: {avg_cont_accuracy:.4f}")
avg_accuracy = (dis_accuracy + cont_accuracy) / len(data.columns)
print(f"Overall Average Accuracy: {avg_accuracy:.4f}")


Feature Column age:
Predicted attribute values: [0.30576608 0.27236685 0.48424324 0.3250105  0.28961667]
True attribute values: [0.47945204 0.10958904 0.20547946 0.1780822  0.19178082]
Accuracy: 0.2060

Feature Column workclass:
Predicted attribute values: [5 5 5 5 5]
True attribute values: [5. 5. 5. 5. 5.]
Accuracy: 0.7178

Feature Column fnlwgt:
Predicted attribute values: [0.11950514 0.13330096 0.10131243 0.12053657 0.12703946]
True attribute values: [0.10296086 0.18163742 0.18393765 0.07988824 0.12894058]
Accuracy: 0.5513

Feature Column education:
Predicted attribute values: [12 16 12 12 12]
True attribute values: [12. 12. 12. 12. 12.]
Accuracy: 0.7433

Feature Column education-num:
Predicted attribute values: [13 16 16 13  5]
True attribute values: [15. 16. 16. 13. 16.]
Accuracy: 0.7248

Feature Column marital-status:
Predicted attribute values: [3 5 1 1 3]
True attribute values: [3. 5. 5. 5. 1.]
Accuracy: 0.7626

Feature Column occupation:
Predicted attribute values: [ 2 13  4 